In [1]:
import bioMass.dataViz as dv
from bioMass.dataloader import SentinelDataset, SentinelDataModule
from bioMass.model import Unet, DoubleStreamUnet
from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm
from torch import nn
import os
import numpy as np
import pandas as pd

from pytorch_lightning import loggers as pl_loggers

%config Completer.use_jedi = False
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
def load_UNetModel(logdir, DataParallel=False):
    "Returns loaded model from checkpoint and hyperparameters"
    if not logdir.endswith("/"):
        logdir = logdir + "/"

    with open(glob(logdir + "hparams.yaml")[0]) as f:
        hparams = yaml.load(f, Loader=yaml.BaseLoader)


    p = float(hparams["dropout_p"])  # dropout (zeroing) probability
    in_channels = int(hparams["in_channels"])
    init_n_features = int(hparams["init_n_features"])

    model = Unet(None, 0, init_n_features, in_channels, p)

    ckpt_file = glob(logdir + "checkpoints/*.ckpt")
    assert len(glob(logdir + "checkpoints/*.ckpt")) == 1
    ckpt_file = ckpt_file[0]

    model = model.load_from_checkpoint(ckpt_file).cuda()
    if DataParallel:
        model = torch.nn.DataParallel(model)

    return model, hparams

In [2]:
dm = SentinelDataModule(max_chips=100, loader_device='cpu', num_workers=1)
dm.setup('fit')

In [4]:
loss_module = nn.MSELoss(reduction='mean')
model = DoubleStreamUnet(loss_fn=loss_module, lr=1e-4, dropout_p=0.5)

/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(


In [5]:
test_dataloader = dm.test_dataloader()

In [5]:
for sample in train_dataloader:
    sample
    break

In [6]:
model(sample['image_s1'], sample['image_s2']).shape

torch.Size([32, 1, 256, 256])